<a href="https://colab.research.google.com/github/emamanic13/mia-13/blob/main/TrabajoFinal_CompEvol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Te damos la bienvenida a Colab

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/notebooks/intro.ipynb
"""

# ============================================================
#  GENERADOR FINAL DE HORARIOS (GA) - TODO EN 1 CELDA (COLAB)
#  - Datos incluidos (61 cursos, 59 docentes) aproximados
#  - DC/TD simulados (para que cuadre)
#  - Inicialización greedy + reparación (repair) -> horarios válidos
#  - Genera N soluciones y elige la mejor
#  - Muestra TODOS los horarios (ciclos 1..10) en tablas tipo Excel
#  - Exporta CSV + ZIP descargable
# ============================================================

import random, csv, os, zipfile
from dataclasses import dataclass
from collections import defaultdict
from typing import Dict, List, Tuple

import pandas as pd
from google.colab import files

# ----------------------------
# CONFIG
# ----------------------------
DAYS = ["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES"]
NUM_PERIODS_PER_DAY = 16

TURN_PERIOD_RANGES = {"M": (1, 8), "N": (9, 16)}
TURN_BY_CYCLE = {c: ("M" if c <= 5 else "N") for c in range(1, 11)}

# ============================================================
# DATOS (incluidos)
# ============================================================
@dataclass(frozen=True)
class Course:
    course_id: int
    code: str
    name: str
    hours_per_week: int
    cycle: int
    students: int = 60

@dataclass(frozen=True)
class Teacher:
    teacher_id: int
    name: str

# 61 cursos (aprox)
COURSES: Dict[int, Course] = {
    1:  Course(1,  "ISI1101", "MATEMATICA I", 5, 1),
    2:  Course(2,  "ISI1102", "TALLER DE LENGUAJE I", 4, 1),
    3:  Course(3,  "ISI1103", "METODOLOGIA DEL TRABAJO UNIVERSITARIO", 4, 1),
    4:  Course(4,  "ISI1104", "FISICA I", 5, 1),
    5:  Course(5,  "ISI1105", "ALGORITMICA I", 6, 1),
    6:  Course(6,  "ISI1106", "ACTIVIDAD A1", 2, 1),

    7:  Course(7,  "ISI1201", "MATEMATICA II", 5, 2),
    8:  Course(8,  "ISI1202", "TALLER DE LENGUAJE II", 4, 2),
    9:  Course(9,  "ISI1203", "ESTADISTICA Y PROBABILIDADES", 5, 2),
    10: Course(10, "ISI1204", "FISICA II", 5, 2),
    11: Course(11, "ISI1205", "ALGORITMICA II", 6, 2),
    12: Course(12, "ISI1206", "ACTIVIDAD B", 2, 2),

    13: Course(13, "ISI1301", "MATEMATICA III", 5, 3),
    14: Course(14, "ISI1302", "ALGORITMICA III", 6, 3),
    15: Course(15, "ISI1303", "ALGEBRA COMPUTACIONAL", 6, 3),
    16: Course(16, "ISI1304", "TALLER DE DISENO GRAFICO", 3, 3),
    17: Course(17, "ISI1305", "ECONOMIA", 2, 3),
    18: Course(18, "ISI1306", "ACTIVIDAD C", 2, 3),

    19: Course(19, "ISI1401", "MATEMATICA IV", 5, 4),
    20: Course(20, "ISI1402", "METODOLOGIA DE LA INVESTIGACION CIENTIFICA", 4, 4),
    21: Course(21, "ISI1403", "INTRODUCCION A LA FILOSOFIA", 4, 4),
    22: Course(22, "ISI1404", "CIRCUITOS DIGITALES", 5, 4),
    23: Course(23, "ISI1405", "ADMINISTRACION Y ORGANIZACION", 3, 4),
    24: Course(24, "ISI1406", "ESTRUCTURA DE DATOS", 4, 4),

    25: Course(25, "ISI1501", "METODOS NUMERICOS", 5, 5),
    26: Course(26, "ISI1502", "VISION HISTORICA DEL PERU Y DEL MUNDO", 2, 5),
    27: Course(27, "ISI1503", "PROCESOS ESTOCASTICOS", 5, 5),
    28: Course(28, "ISI1504", "SISTEMAS DIGITALES", 5, 5),
    29: Course(29, "ISI1505", "INGENIERIA DE SOFTWARE I", 5, 5),
    30: Course(30, "ISI1506", "COMPUTACION GRAFICA", 5, 5),
    31: Course(31, "ISI1507", "BASE DE DATOS I", 5, 5),

    32: Course(32, "ISI1601", "FILOSOFIA CIENCIA Y TECNOLOGIA", 4, 6),
    33: Course(33, "ISI1602", "TELECOMUNICACIONES", 5, 6),
    34: Course(34, "ISI1603", "INGENIERIA DE SOFTWARE II", 5, 6),
    35: Course(35, "ISI1604", "ARQUITECTURA DE COMPUTADORAS", 4, 6),
    36: Course(36, "ISI1605", "INVESTIGACION DE OPERACIONES I", 5, 6),
    37: Course(37, "ISI1606", "BASE DE DATOS II", 5, 6),

    38: Course(38, "ISI1701", "INGENIERIA DE CONTROL", 5, 7),
    39: Course(39, "ISI1702", "INGENIERIA DE SOFTWARE III", 6, 7),
    40: Course(40, "ISI1703", "SISTEMAS OPERATIVOS", 5, 7),
    41: Course(41, "ISI1704", "INVESTIGACION DE OPERACIONES II", 5, 7),
    42: Course(42, "ISI1705", "INGENIERIA DE NEGOCIOS", 2, 7),
    43: Course(43, "ISI1706", "ETICA PROFESIONAL", 2, 7),

    44: Course(44, "ISI1801", "REDES TELEMATICAS", 5, 8),
    45: Course(45, "ISI1802", "INTELIGENCIA ARTIFICIAL I", 5, 8),
    46: Course(46, "ISI1803", "SIMULACION", 5, 8),
    47: Course(47, "ISI1804", "FORMULACION Y EVALUACION DE PROYECTOS", 3, 8),
    48: Course(48, "ISI1805", "MARKETING", 3, 8),

    49: Course(49, "ISI1901", "TALLER DE TESIS I", 5, 9),
    50: Course(50, "ISI1902", "PRACTICA PROFESIONAL I", 3, 9),
    51: Course(51, "ISI1903", "INTELIGENCIA ARTIFICIAL II", 2, 9),
    52: Course(52, "ISI1904", "SISTEMAS DISTRIBUIDOS", 5, 9),
    53: Course(53, "ISI1905", "GESTION DE PROYECTOS INFORMATICOS", 3, 9),
    54: Course(54, "ISI1906", "DERECHO EMPRESARIAL E INFORMATICO", 3, 9),
    55: Course(55, "ISI1907", "ELECTIVO I", 2, 9),

    56: Course(56, "ISI2001", "TALLER DE TESIS II", 5, 10),
    57: Course(57, "ISI2002", "PRACTICA PROFESIONAL II", 3, 10),
    58: Course(58, "ISI2003", "AUDITORIA TI", 3, 10),
    59: Course(59, "ISI2004", "GERENCIA INFORMATICA", 3, 10),
    60: Course(60, "ISI2005", "ELECTIVO II", 3, 10),
    61: Course(61, "ISI2006", "ELECTIVO III", 3, 10),
}

TEACHERS: Dict[int, Teacher] = {i: Teacher(i, f"DOCENTE_{i:02d}") for i in range(1, 60)}

# ============================================================
# SESIONES (consecutivas)
# ============================================================
@dataclass(frozen=True)
class Session:
    session_id: int
    course_id: int
    length: int

@dataclass
class Gene:
    teacher_id: int
    day: int
    start_period: int

def split_into_sessions(hours: int) -> List[int]:
    # 2+2+1, 2+2+2, etc.
    res = []
    while hours >= 2:
        res.append(2)
        hours -= 2
    if hours == 1:
        res.append(1)
    return res

def build_sessions(courses: Dict[int, Course]) -> List[Session]:
    sessions = []
    sid = 1
    for c in courses.values():
        for length in split_into_sessions(c.hours_per_week):
            sessions.append(Session(sid, c.course_id, length))
            sid += 1
    return sessions

# ============================================================
# DC / TD (simulados para que cuadre)
# ============================================================
def gen_preferences_dc(seed: int) -> Dict[int, Dict[int, int]]:
    rng = random.Random(seed)
    dc = {tid: {} for tid in TEACHERS.keys()}
    tids = list(TEACHERS.keys())
    for c in COURSES.values():
        # más candidatos => más factible
        k = rng.randint(6, 12)
        chosen = rng.sample(tids, k=k)
        for tid in chosen:
            dc[tid][c.course_id] = 1
    return dc

def gen_availability_td(seed: int) -> Dict[int, List[List[int]]]:
    rng = random.Random(seed)
    td = {}
    for tid in TEACHERS.keys():
        td[tid] = [[0]*(NUM_PERIODS_PER_DAY+1) for _ in range(len(DAYS))]
        roll = rng.random()
        if roll < 0.45:
            pmin, pmax = TURN_PERIOD_RANGES["M"]
        elif roll < 0.90:
            pmin, pmax = TURN_PERIOD_RANGES["N"]
        else:
            pmin, pmax = (1, 16)

        for d in range(len(DAYS)):
            for p in range(pmin, pmax+1):
                td[tid][d][p] = 1

        # recortes suaves
        for _ in range(rng.randint(1, 6)):
            d = rng.randrange(len(DAYS))
            p = rng.randrange(1, NUM_PERIODS_PER_DAY+1)
            td[tid][d][p] = 0
    return td

def can_teach(dc, tid, cid):
    return dc.get(tid, {}).get(cid, 0) == 1

def teacher_available(td, tid, day, start, length) -> bool:
    for p in range(start, start+length):
        if p < 1 or p > NUM_PERIODS_PER_DAY:
            return False
        if td[tid][day][p] != 1:
            return False
    return True

def session_in_turn(course: Course, start: int, length: int) -> bool:
    t = TURN_BY_CYCLE[course.cycle]
    pmin, pmax = TURN_PERIOD_RANGES[t]
    return start >= pmin and (start+length-1) <= pmax

# ============================================================
# FITNESS (penaliza huecos; evita choques)
# ============================================================
def gaps_penalty(assigns, courses):
    occ = defaultdict(lambda: [set() for _ in range(len(DAYS))])
    for cid, day, start, length in assigns:
        cyc = courses[cid].cycle
        for p in range(start, start+length):
            occ[cyc][day].add(p)

    gaps = 0
    for cyc, day_sets in occ.items():
        for d in range(len(DAYS)):
            used = day_sets[d]
            if not used:
                continue
            mn, mx = min(used), max(used)
            gaps += sum(1 for p in range(mn, mx+1) if p not in used)
    return gaps

def fitness(ind, sessions, courses, dc, td):
    HARD = 100000
    PREF = 15000
    AVAIL = 15000
    TURN = 8000
    GAPW = 20

    teacher_time = defaultdict(set)
    cycle_time = defaultdict(set)
    assigns = []
    pen = 0

    for gene, s in zip(ind, sessions):
        c = courses[s.course_id]

        if not can_teach(dc, gene.teacher_id, c.course_id):
            pen += PREF
        if not teacher_available(td, gene.teacher_id, gene.day, gene.start_period, s.length):
            pen += AVAIL
        if not session_in_turn(c, gene.start_period, s.length):
            pen += TURN

        kt = (gene.teacher_id, gene.day)
        for p in range(gene.start_period, gene.start_period+s.length):
            if p in teacher_time[kt]:
                pen += HARD
            teacher_time[kt].add(p)

        kc = (c.cycle, gene.day)
        for p in range(gene.start_period, gene.start_period+s.length):
            if p in cycle_time[kc]:
                pen += HARD
            cycle_time[kc].add(p)

        assigns.append((c.course_id, gene.day, gene.start_period, s.length))

    pen += gaps_penalty(assigns, courses) * GAPW
    return pen

# ============================================================
# GREEDY + REPAIR (clave para que cuadre)
# ============================================================
def greedy_gene_for_session(s, courses, dc, td, teacher_time, cycle_time, tries=3000):
    c = courses[s.course_id]
    turn = TURN_BY_CYCLE[c.cycle]
    pmin, pmax = TURN_PERIOD_RANGES[turn]
    latest = pmax - s.length + 1

    candidates = [tid for tid in TEACHERS.keys() if can_teach(dc, tid, c.course_id)]
    if not candidates:
        candidates = list(TEACHERS.keys())

    for _ in range(tries):
        tid = random.choice(candidates)
        day = random.randrange(len(DAYS))
        start = random.randint(pmin, latest)

        if not teacher_available(td, tid, day, start, s.length):
            continue

        conflict = False
        for p in range(start, start+s.length):
            if p in teacher_time[(tid, day)]:
                conflict = True; break
            if p in cycle_time[(c.cycle, day)]:
                conflict = True; break
        if conflict:
            continue

        # reservar
        for p in range(start, start+s.length):
            teacher_time[(tid, day)].add(p)
            cycle_time[(c.cycle, day)].add(p)

        return Gene(tid, day, start)

    # fallback (raro con tries alto)
    tid = random.choice(list(TEACHERS.keys()))
    day = random.randrange(len(DAYS))
    start = random.randint(1, NUM_PERIODS_PER_DAY - s.length + 1)
    return Gene(tid, day, start)

def build_greedy_individual(sessions, courses, dc, td):
    teacher_time = defaultdict(set)
    cycle_time = defaultdict(set)
    ind = []
    for s in sessions:
        ind.append(greedy_gene_for_session(s, courses, dc, td, teacher_time, cycle_time))
    return ind

def build_initial_population(pop_size, sessions, courses, dc, td):
    return [build_greedy_individual(sessions, courses, dc, td) for _ in range(pop_size)]

def repair(ind, sessions, courses, dc, td, max_passes=4):
    for _ in range(max_passes):
        teacher_time = defaultdict(set)
        cycle_time = defaultdict(set)
        new_ind = []
        for gene, s in zip(ind, sessions):
            c = courses[s.course_id]

            ok_pref = can_teach(dc, gene.teacher_id, c.course_id)
            ok_av = teacher_available(td, gene.teacher_id, gene.day, gene.start_period, s.length)
            ok_turn = session_in_turn(c, gene.start_period, s.length)

            conflict = False
            if ok_av and ok_turn:
                for p in range(gene.start_period, gene.start_period+s.length):
                    if p in teacher_time[(gene.teacher_id, gene.day)]:
                        conflict = True; break
                    if p in cycle_time[(c.cycle, gene.day)]:
                        conflict = True; break
            else:
                conflict = True

            if (not ok_pref) or conflict:
                gene = greedy_gene_for_session(s, courses, dc, td, teacher_time, cycle_time, tries=3000)
            else:
                for p in range(gene.start_period, gene.start_period+s.length):
                    teacher_time[(gene.teacher_id, gene.day)].add(p)
                    cycle_time[(c.cycle, gene.day)].add(p)

            new_ind.append(gene)
        ind = new_ind
    return ind

# ============================================================
# GA
# ============================================================
def tournament(pop, scores, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[min(idxs, key=lambda i: scores[i])]

def crossover(a, b):
    c1, c2 = [], []
    for ga, gb in zip(a, b):
        if random.random() < 0.5:
            c1.append(Gene(ga.teacher_id, ga.day, ga.start_period))
            c2.append(Gene(gb.teacher_id, gb.day, gb.start_period))
        else:
            c1.append(Gene(gb.teacher_id, gb.day, gb.start_period))
            c2.append(Gene(ga.teacher_id, ga.day, ga.start_period))
    return c1, c2

def mutate(ind, sessions, courses, dc, td, rate=0.06):
    for i in range(len(ind)):
        if random.random() < rate:
            teacher_time = defaultdict(set)
            cycle_time = defaultdict(set)
            # reconstruye ocupación sin el gen i
            for j, (g, s) in enumerate(zip(ind, sessions)):
                if j == i:
                    continue
                c = courses[s.course_id]
                for p in range(g.start_period, g.start_period+s.length):
                    teacher_time[(g.teacher_id, g.day)].add(p)
                    cycle_time[(c.cycle, g.day)].add(p)
            ind[i] = greedy_gene_for_session(sessions[i], courses, dc, td, teacher_time, cycle_time, tries=2500)

def elitism(pop, scores, e=10):
    idxs = sorted(range(len(pop)), key=lambda i: scores[i])
    return [pop[i] for i in idxs[:e]]

def run_ga(seed, pop_size=180, generations=500, elite_size=10, cross=0.85, mut=0.06):
    random.seed(seed)
    sessions = build_sessions(COURSES)
    dc = gen_preferences_dc(seed)
    td = gen_availability_td(seed+999)

    pop = build_initial_population(pop_size, sessions, COURSES, dc, td)

    best_ind, best_score = None, None

    for g in range(1, generations+1):
        scores = [fitness(ind, sessions, COURSES, dc, td) for ind in pop]
        bi = min(range(len(pop)), key=lambda i: scores[i])

        if best_score is None or scores[bi] < best_score:
            best_score = scores[bi]
            best_ind = pop[bi]

        if g in (1, generations) or g % max(1, generations//10) == 0:
            print(f"[seed={seed}] Gen {g}/{generations} best={scores[bi]} global_best={best_score}")

        new_pop = elitism(pop, scores, elite_size)

        while len(new_pop) < pop_size:
            p1 = tournament(pop, scores, 3)
            p2 = tournament(pop, scores, 3)

            if random.random() < cross:
                c1, c2 = crossover(p1, p2)
            else:
                c1 = [Gene(x.teacher_id, x.day, x.start_period) for x in p1]
                c2 = [Gene(x.teacher_id, x.day, x.start_period) for x in p2]

            mutate(c1, sessions, COURSES, dc, td, mut)
            mutate(c2, sessions, COURSES, dc, td, mut)

            c1 = repair(c1, sessions, COURSES, dc, td, max_passes=4)
            c2 = repair(c2, sessions, COURSES, dc, td, max_passes=4)

            new_pop.append(c1)
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        pop = new_pop

    return best_ind, best_score, sessions

# ============================================================
# EXPORT + MOSTRAR EN COLAB
# ============================================================
def build_timetable(ind, sessions):
    # cycle -> day -> period -> str
    table = {}
    for c in COURSES.values():
        if c.cycle not in table:
            table[c.cycle] = [["" for _ in range(NUM_PERIODS_PER_DAY+1)] for _ in range(len(DAYS))]

    for gene, s in zip(ind, sessions):
        c = COURSES[s.course_id]
        t = TEACHERS[gene.teacher_id].name
        label = f"{c.code} | {t}"
        for p in range(gene.start_period, gene.start_period+s.length):
            # si alguna vez chocara (raro), concatena en vez de pisar
            if table[c.cycle][gene.day][p] == "":
                table[c.cycle][gene.day][p] = label
            else:
                table[c.cycle][gene.day][p] = table[c.cycle][gene.day][p] + " || " + label
    return table

def timetable_to_df(tt, cycle, only_course=True):
    rows = []
    for p in range(1, NUM_PERIODS_PER_DAY+1):
        row = {"PERIODO": p}
        for d, dayname in enumerate(DAYS):
            cell = tt[cycle][d][p]
            if cell == "":
                cell = ""
            else:
                if only_course:
                    # solo el código del curso (más fácil de leer)
                    cell = cell.split("|")[0].strip()
            row[dayname] = cell
        rows.append(row)
    return pd.DataFrame(rows)

def export_cycle_csv(tt_cycle, path):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["PERIODO"] + DAYS)
        for p in range(1, NUM_PERIODS_PER_DAY+1):
            w.writerow([p] + [tt_cycle[d][p] for d in range(len(DAYS))])

def export_all_cycles(tt, outdir, tag):
    os.makedirs(outdir, exist_ok=True)
    for cyc in sorted(tt.keys()):
        export_cycle_csv(tt[cyc], os.path.join(outdir, f"{tag}_ciclo_{cyc}.csv"))

def zip_folder(folder, zipname):
    with zipfile.ZipFile(zipname, "w", compression=zipfile.ZIP_DEFLATED) as z:
        for root, _, files_ in os.walk(folder):
            for fn in files_:
                full = os.path.join(root, fn)
                rel = os.path.relpath(full, folder)
                z.write(full, rel)

def check_hours(ind, sessions):
    # verifica que cada curso tenga exactamente sus horas
    assigned = defaultdict(int)
    for gene, s in zip(ind, sessions):
        assigned[s.course_id] += s.length

    mism = []
    for cid, c in COURSES.items():
        exp = c.hours_per_week
        got = assigned.get(cid, 0)
        if exp != got:
            mism.append((c.code, exp, got, got-exp))
    return mism

# ============================================================
# PARAMETROS (cambia aquí si quieres)
# ============================================================
OUTDIR = "salida_horarios"
ZIP_NAME = "horarios_finales.zip"

SOLUTIONS = 3       # cuántas corridas (más = más chances de un mejor horario)
BASE_SEED = 277     # semilla base
POP = 180
GEN = 500

# ============================================================
# EJECUCIÓN: genera N soluciones y elige la mejor
# ============================================================
results = []
for i in range(SOLUTIONS):
    seed = BASE_SEED + i*77
    ind, score, sessions = run_ga(seed, pop_size=POP, generations=GEN)
    results.append((score, seed, ind, sessions))

results.sort(key=lambda x: x[0])
best_score, best_seed, best_ind, best_sessions = results[0]

print("\n==============================")
print("  MEJOR SOLUCIÓN ENCONTRADA")
print("  seed:", best_seed)
print("  fitness:", best_score)
print("==============================\n")

# Verificación de horas
mism = check_hours(best_ind, best_sessions)
if not mism:
    print("✅ Verificación horas: OK (todas las horas cuadran)")
else:
    print("⚠️ Verificación horas: HAY DIFERENCIAS (raro). Detalle:")
    for x in mism[:20]:
        print(x)

# Construye horario final
best_tt = build_timetable(best_ind, best_sessions)

# ------------------------------------------------------------
# 1) MUESTRA TODOS LOS HORARIOS EN COLAB (TABLAS TIPO EXCEL)
# ------------------------------------------------------------
print("\n📌 MOSTRANDO TODOS LOS CICLOS (1..10) EN TABLAS\n")
for cycle in range(1, 11):
    df = timetable_to_df(best_tt, cycle, only_course=True)
    print(f"🟦 HORARIO CICLO {cycle} (solo cursos) 🟦")
    display(df)

# Si quieres ver "curso + docente" en vez de solo curso:
# cambia only_course=False:
# display(timetable_to_df(best_tt, cycle, only_course=False))

# ------------------------------------------------------------
# 2) EXPORTA CSV + ZIP (descargable)
# ------------------------------------------------------------
# limpia carpeta
if os.path.exists(OUTDIR):
    for root, _, files_ in os.walk(OUTDIR):
        for fn in files_:
            os.remove(os.path.join(root, fn))
else:
    os.makedirs(OUTDIR, exist_ok=True)

tag = f"seed_{best_seed}_fit_{best_score}"
export_all_cycles(best_tt, OUTDIR, tag)
zip_folder(OUTDIR, ZIP_NAME)

print(f"\n✅ CSV exportados en: {OUTDIR}")
print(f"✅ ZIP creado: {ZIP_NAME}")

files.download(ZIP_NAME)

# ============================================
# ✅ HORARIOS CON ALGORITMO GENÉTICO (TODO JUNTO)
# - Corre en Google Colab en UNA sola celda
# - Incluye imports, datos (aprox), GA, verificación, impresión legible
# - Exporta CSV por ciclo y crea ZIP final
# ============================================

import os, csv, zipfile, random
import numpy as np

# =========================
# CONFIGURACIÓN GENERAL
# =========================
DIAS = ["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES"]
N_DIAS = 5
PERIODOS = list(range(1, 17))  # 1..16

# Turnos por ciclo (como en tu salida)
TURNOS = {
    # ciclos 1..5: mañana (1..8)
    1: (1, 8), 2: (1, 8), 3: (1, 8), 4: (1, 8), 5: (1, 8),
    # ciclos 6..10: noche (9..16)
    6: (9, 16), 7: (9, 16), 8: (9, 16), 9: (9, 16), 10: (9, 16),
}

# GA
POP = 180
GEN = 500
SEEDS = [277, 354, 431]
PATIENCE = 80        # early stop si no mejora
PRINT_EVERY = 50     # log cada N gen
MUT_RATE = 0.18
ELITE = 6
TOURN = 4

OUTDIR = "salida_horarios"
ZIP_NAME = "horarios_finales.zip"


# =========================
# DATOS (APROXIMADOS) - 10 ciclos
# (Para práctica: códigos como en tu horario)
# Ajusta horas si quieres, pero esto corre ya.
# =========================
# Nota: horas típicas 2/3/4/5. Esto es "aprox" para que cuadre y genere horarios.
CICLOS_CURSOS = {
    1: ["ISI1101","ISI1102","ISI1103","ISI1104","ISI1105","ISI1106"],
    2: ["ISI1201","ISI1202","ISI1203","ISI1204","ISI1205","ISI1206"],
    3: ["ISI1301","ISI1302","ISI1303","ISI1304","ISI1305","ISI1306"],
    4: ["ISI1401","ISI1402","ISI1403","ISI1404","ISI1405","ISI1406"],
    5: ["ISI1501","ISI1502","ISI1503","ISI1504","ISI1505","ISI1506","ISI1507"],
    6: ["ISI1601","ISI1602","ISI1603","ISI1604","ISI1605","ISI1606"],
    7: ["ISI1701","ISI1702","ISI1703","ISI1704","ISI1705","ISI1706"],
    8: ["ISI1801","ISI1802","ISI1803","ISI1804","ISI1805"],
    9: ["ISI1901","ISI1902","ISI1903","ISI1904","ISI1905","ISI1906","ISI1907"],
    10:["ISI2001","ISI2002","ISI2003","ISI2004","ISI2005","ISI2006"],
}

# Horas por curso (aprox). Si tienes las reales, cámbialas aquí.
HORAS_CURSO = {
    # ciclo 1
    "ISI1101":4,"ISI1102":4,"ISI1103":3,"ISI1104":4,"ISI1105":4,"ISI1106":2,
    # ciclo 2
    "ISI1201":4,"ISI1202":4,"ISI1203":4,"ISI1204":4,"ISI1205":4,"ISI1206":2,
    # ciclo 3
    "ISI1301":4,"ISI1302":4,"ISI1303":4,"ISI1304":3,"ISI1305":3,"ISI1306":2,
    # ciclo 4
    "ISI1401":4,"ISI1402":4,"ISI1403":3,"ISI1404":4,"ISI1405":3,"ISI1406":4,
    # ciclo 5
    "ISI1501":4,"ISI1502":2,"ISI1503":4,"ISI1504":5,"ISI1505":4,"ISI1506":3,"ISI1507":4,
    # ciclo 6
    "ISI1601":4,"ISI1602":4,"ISI1603":3,"ISI1604":3,"ISI1605":4,"ISI1606":4,
    # ciclo 7
    "ISI1701":4,"ISI1702":4,"ISI1703":4,"ISI1704":4,"ISI1705":2,"ISI1706":2,
    # ciclo 8
    "ISI1801":4,"ISI1802":4,"ISI1803":4,"ISI1804":3,"ISI1805":2,
    # ciclo 9
    "ISI1901":5,"ISI1902":3,"ISI1903":2,"ISI1904":4,"ISI1905":4,"ISI1906":3,"ISI1907":2,
    # ciclo 10
    "ISI2001":4,"ISI2002":3,"ISI2003":3,"ISI2004":2,"ISI2005":3,"ISI2006":3,
}

# docentes (simulados)
N_DOCENTES = 59
DOCENTES = [f"DOC_{i:02d}" for i in range(1, N_DOCENTES+1)]

# asignación curso->lista de docentes posibles (simulada)
random.seed(1)
CURSO_DOCENTES = {c: random.sample(DOCENTES, k=8) for ciclo in CICLOS_CURSOS for c in CICLOS_CURSOS[ciclo]}

# disponibilidad docente por turno (simulada)
# - algunos solo mañana, otros solo noche, otros mixto
random.seed(2)
DISP_DOCENTE = {}
for d in DOCENTES:
    r = random.random()
    if r < 0.35:
        DISP_DOCENTE[d] = ("MANANA",)  # solo periodos 1..8
    elif r < 0.70:
        DISP_DOCENTE[d] = ("NOCHE",)   # solo periodos 9..16
    else:
        DISP_DOCENTE[d] = ("MANANA","NOCHE")


def docente_disponible(doc, periodo, ciclo):
    # turno por ciclo
    a,b = TURNOS[ciclo]
    if not (a <= periodo <= b):
        return False
    t = "MANANA" if periodo <= 8 else "NOCHE"
    return t in DISP_DOCENTE[doc]


# =========================
# MODELADO DE SESIONES
# =========================
# Representación de una sesión:
# (curso, ciclo, docente, dia_idx, periodo_inicio, duracion)

def dividir_horas(h):
    # preferimos bloques de 2 horas, y si queda 1 -> bloque de 1
    res = []
    while h >= 2:
        res.append(2)
        h -= 2
    if h == 1:
        res.append(1)
    return res

# Preconstruimos la lista total de "sesiones a ubicar"
SESIONES = []
for ciclo, cursos in CICLOS_CURSOS.items():
    for c in cursos:
        for dur in dividir_horas(HORAS_CURSO[c]):
            SESIONES.append((c, ciclo, dur))

# índice para acceso rápido por curso
CURSO_A_CICLO = {}
for ciclo, cursos in CICLOS_CURSOS.items():
    for c in cursos:
        CURSO_A_CICLO[c] = ciclo


# =========================
# UTILIDADES DE GRILLA
# =========================
def slots_de_sesion(dia, start, dur):
    return [(dia, p) for p in range(start, start+dur)]

def turno_rango(ciclo):
    return TURNOS[ciclo]

def random_posicion_valida(ciclo, dur):
    a,b = turno_rango(ciclo)
    # inicio debe permitir duracion
    start = random.randint(a, b - dur + 1)
    dia = random.randint(0, N_DIAS-1)
    return dia, start

def construir_grillas(individuo):
    """
    Devuelve:
      grid_ciclo[ciclo][dia][periodo] = curso o "-"
      grid_doc[doc][dia][periodo] = curso o "-"
    """
    grid_ciclo = {c: {d: {p: "-" for p in PERIODOS} for d in range(N_DIAS)} for c in range(1,11)}
    grid_doc = {doc: {d: {p: "-" for p in PERIODOS} for d in range(N_DIAS)} for doc in DOCENTES}
    for (curso, ciclo, doc, dia, start, dur) in individuo:
        for (_, p) in slots_de_sesion(dia, start, dur):
            grid_ciclo[ciclo][dia][p] = curso
            grid_doc[doc][dia][p] = curso
    return grid_ciclo, grid_doc

def verificar_horas(individuo):
    """
    Verifica que cada curso tenga exactamente HORAS_CURSO[curso] periodos asignados.
    """
    cont = {c:0 for c in HORAS_CURSO}
    for (curso, ciclo, doc, dia, start, dur) in individuo:
        cont[curso] += dur
    ok = all(cont[c] == HORAS_CURSO[c] for c in cont)
    return ok, cont


# =========================
# CREAR POBLACIÓN INICIAL (greedy con reintentos)
# =========================
def crear_individuo():
    indiv = []
    # estructuras para evitar choques dentro de ciclo y docente en construcción inicial
    ocup_ciclo = {c: set() for c in range(1,11)}   # (dia,periodo)
    ocup_doc = {d: set() for d in DOCENTES}        # (dia,periodo)

    for (curso, ciclo, dur) in SESIONES:
        # elegimos un docente posible
        docentes_pos = CURSO_DOCENTES[curso]
        doc = random.choice(docentes_pos)

        # reintentos para ubicar sin choques básicos
        placed = False
        for _ in range(250):
            # podemos cambiar docente en reintentos
            doc_try = doc if random.random() < 0.75 else random.choice(docentes_pos)
            dia, start = random_posicion_valida(ciclo, dur)
            ss = slots_de_sesion(dia, start, dur)

            # reglas duras en construcción
            # - dentro del turno del ciclo
            a,b = turno_rango(ciclo)
            if start < a or (start+dur-1) > b:
                continue

            # - disponibilidad docente
            if not all(docente_disponible(doc_try, p, ciclo) for (_,p) in ss):
                continue

            # - no choque en ciclo
            if any((dia,p) in ocup_ciclo[ciclo] for (_,p) in ss):
                continue

            # - no choque docente
            if any((dia,p) in ocup_doc[doc_try] for (_,p) in ss):
                continue

            # colocar
            indiv.append((curso, ciclo, doc_try, dia, start, dur))
            for (_,p) in ss:
                ocup_ciclo[ciclo].add((dia,p))
                ocup_doc[doc_try].add((dia,p))
            placed = True
            break

        # si no pudo ubicarse, lo ponemos “como sea” (será penalizado y luego el GA lo arregla)
        if not placed:
            doc_try = random.choice(docentes_pos)
            dia, start = random_posicion_valida(ciclo, dur)
            indiv.append((curso, ciclo, doc_try, dia, start, dur))

    return indiv

def crear_poblacion():
    return [crear_individuo() for _ in range(POP)]


# =========================
# FITNESS (penalizaciones)
# =========================
def fitness(individuo):
    """
    Menor = mejor.
    Penaliza:
      - choques en ciclo (duro)
      - choques de docente (duro)
      - fuera de turno (duro)
      - docente no disponible (duro)
      - huecos dentro del día por ciclo (blando)
    """
    pen = 0

    # ocupar por ciclo y docente
    ocup_ciclo = {c: {} for c in range(1,11)}  # (dia,periodo)->count
    ocup_doc = {d: {} for d in DOCENTES}

    # duros
    for (curso, ciclo, doc, dia, start, dur) in individuo:
        # turno
        a,b = turno_rango(ciclo)
        if start < a or (start+dur-1) > b:
            pen += 4000

        # disponibilidad
        for (_,p) in slots_de_sesion(dia, start, dur):
            if not docente_disponible(doc, p, ciclo):
                pen += 2500

            ocup_ciclo[ciclo][(dia,p)] = ocup_ciclo[ciclo].get((dia,p),0) + 1
            ocup_doc[doc][(dia,p)] = ocup_doc[doc].get((dia,p),0) + 1

    # choques
    for ciclo in range(1,11):
        for k,v in ocup_ciclo[ciclo].items():
            if v > 1:
                pen += 6000*(v-1)

    for doc in DOCENTES:
        for k,v in ocup_doc[doc].items():
            if v > 1:
                pen += 6000*(v-1)

    # blandos: huecos por ciclo (dentro de su turno)
    grid_ciclo, _ = construir_grillas(individuo)
    for ciclo in range(1,11):
        a,b = turno_rango(ciclo)
        for dia in range(N_DIAS):
            # lista de ocupación en el turno
            occ = [1 if grid_ciclo[ciclo][dia][p] != "-" else 0 for p in range(a,b+1)]
            if sum(occ) == 0:
                continue
            # huecos internos: 1 0 1 dentro del bloque
            first = occ.index(1)
            last = len(occ)-1 - occ[::-1].index(1)
            inside = occ[first:last+1]
            gaps = inside.count(0)
            pen += 20*gaps

    return pen


# =========================
# OPERADORES GA
# =========================
def torneo(poblacion, fits):
    # selecciona el mejor de TOURN individuos al azar
    best_i = None
    best_f = None
    for _ in range(TOURN):
        i = random.randrange(len(poblacion))
        f = fits[i]
        if best_i is None or f < best_f:
            best_i, best_f = i, f
    return poblacion[best_i]

def crossover(a, b):
    # cruce: intercambia un bloque de sesiones (misma longitud)
    n = len(a)
    if n < 2:
        return a[:], b[:]
    i = random.randint(0, n-2)
    j = random.randint(i+1, n-1)
    ca = a[:i] + b[i:j] + a[j:]
    cb = b[:i] + a[i:j] + b[j:]
    return ca, cb

def mutate(ind):
    # mutación: cambia día/hora/docente de algunas sesiones
    out = list(ind)
    for idx in range(len(out)):
        if random.random() < MUT_RATE:
            curso, ciclo, doc, dia, start, dur = out[idx]
            # a veces cambia docente
            if random.random() < 0.35:
                doc = random.choice(CURSO_DOCENTES[curso])
            # cambia posición
            dia, start = random_posicion_valida(ciclo, dur)
            out[idx] = (curso, ciclo, doc, dia, start, dur)
    return out

def repair(ind):
    # Reparación ligera: intenta quitar choques dentro del ciclo (principalmente),
    # recolocando sesiones problemáticas.
    indiv = list(ind)

    # contar ocupación por ciclo
    ocup_ciclo = {c: {} for c in range(1,11)}
    ocup_doc = {d: {} for d in DOCENTES}

    for idx,(curso,ciclo,doc,dia,start,dur) in enumerate(indiv):
        for (_,p) in slots_de_sesion(dia,start,dur):
            ocup_ciclo[ciclo][(dia,p)] = ocup_ciclo[ciclo].get((dia,p),[]) + [idx]
            ocup_doc[doc][(dia,p)] = ocup_doc[doc].get((dia,p),[]) + [idx]

    # detecta indices conflictivos
    conflict = set()
    for ciclo in range(1,11):
        for k, lst in ocup_ciclo[ciclo].items():
            if len(lst) > 1:
                conflict.update(lst[1:])  # deja uno, mueve el resto
    for doc in DOCENTES:
        for k, lst in ocup_doc[doc].items():
            if len(lst) > 1:
                conflict.update(lst[1:])

    if not conflict:
        return indiv

    # reconstruye ocupación actual (sets) para check rápido
    occC = {c:set() for c in range(1,11)}
    occD = {d:set() for d in DOCENTES}
    for idx,(curso,ciclo,doc,dia,start,dur) in enumerate(indiv):
        if idx in conflict:
            continue
        for (_,p) in slots_de_sesion(dia,start,dur):
            occC[ciclo].add((dia,p))
            occD[doc].add((dia,p))

    # recoloca conflictivos
    for idx in list(conflict):
        curso,ciclo,doc,dia,start,dur = indiv[idx]
        docentes_pos = CURSO_DOCENTES[curso]
        placed = False
        for _ in range(200):
            doc_try = doc if random.random() < 0.6 else random.choice(docentes_pos)
            dia_try, start_try = random_posicion_valida(ciclo, dur)
            ss = slots_de_sesion(dia_try, start_try, dur)
            a,b = turno_rango(ciclo)
            if start_try < a or (start_try+dur-1) > b:
                continue
            if not all(docente_disponible(doc_try, p, ciclo) for (_,p) in ss):
                continue
            if any((dia_try,p) in occC[ciclo] for (_,p) in ss):
                continue
            if any((dia_try,p) in occD[doc_try] for (_,p) in ss):
                continue
            # ok
            indiv[idx] = (curso,ciclo,doc_try,dia_try,start_try,dur)
            for (_,p) in ss:
                occC[ciclo].add((dia_try,p))
                occD[doc_try].add((dia_try,p))
            placed = True
            break
        # si no pudo, se queda como está (penalizará)
        if not placed:
            pass

    return indiv

def siguiente_generacion(poblacion, fits):
    # elitismo
    idxs = np.argsort(fits)
    nueva = [poblacion[i] for i in idxs[:ELITE]]

    # relleno
    while len(nueva) < POP:
        p1 = torneo(poblacion, fits)
        p2 = torneo(poblacion, fits)
        h1, h2 = crossover(p1, p2)
        h1 = mutate(h1)
        h2 = mutate(h2)
        h1 = repair(h1)
        h2 = repair(h2)
        nueva.append(h1)
        if len(nueva) < POP:
            nueva.append(h2)

    return nueva


# =========================
# CONSTRUIR HORARIOS PARA MOSTRAR
# =========================
def individuo_a_horarios(individuo):
    # horarios[ciclo][periodo][dia] = curso o "-"
    horarios = {c: {p: {d: "-" for d in DIAS} for p in PERIODOS} for c in range(1,11)}

    for (curso, ciclo, doc, dia_idx, start, dur) in individuo:
        for (_,p) in slots_de_sesion(dia_idx, start, dur):
            horarios[ciclo][p][DIAS[dia_idx]] = curso
    return {"horarios": horarios}

def print_tabla_ciclo(solucion, ciclo):
    print("\n" + "="*48)
    print(f"🟦 HORARIO CICLO {ciclo} (solo cursos) 🟦")
    print("="*48)
    header = f"{'PER':<4}" + "".join([f"{d:<16}" for d in DIAS])
    print(header)
    print("-"*len(header))

    for p in PERIODOS:
        line = f"{p:<4}"
        for d in DIAS:
            val = solucion["horarios"][ciclo][p][d]
            if not val:
                val = "-"
            line += f"{val[:14]:<16}"
        print(line)

def mostrar_todos(solucion):
    for ciclo in range(1,11):
        print_tabla_ciclo(solucion, ciclo)


# =========================
# EXPORT CSV + ZIP
# =========================
def exportar_csv(solucion, outdir=OUTDIR):
    os.makedirs(outdir, exist_ok=True)
    for ciclo in range(1,11):
        path = os.path.join(outdir, f"ciclo_{ciclo}.csv")
        with open(path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["PERIODO"] + DIAS)
            for p in PERIODOS:
                w.writerow([p] + [solucion["horarios"][ciclo][p][d] for d in DIAS])

def crear_zip(outdir=OUTDIR, zipname=ZIP_NAME):
    with zipfile.ZipFile(zipname, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files in os.walk(outdir):
            for fn in files:
                full = os.path.join(root, fn)
                z.write(full, arcname=os.path.relpath(full, outdir))


# =========================
# EJECUCIÓN PRINCIPAL (3 seeds)
# =========================
def run():
    mejor_ind = None
    mejor_fit = float("inf")
    mejor_seed = None

    for seed in SEEDS:
        random.seed(seed)
        np.random.seed(seed)

        poblacion = crear_poblacion()

        best_local = float("inf")
        sin_mejora = 0

        for g in range(1, GEN+1):
            fits = [fitness(ind) for ind in poblacion]
            best_g = min(fits)

            # tracking
            if best_g < best_local:
                best_local = best_g
                sin_mejora = 0
            else:
                sin_mejora += 1

            if g == 1 or g % PRINT_EVERY == 0 or g == GEN:
                print(f"[seed={seed}] Gen {g}/{GEN} best={best_g} global_best={min(mejor_fit, best_local)}")

            # early stop
            if sin_mejora >= PATIENCE:
                print(f"⏹ Early stop seed={seed} en Gen {g} (sin mejora {PATIENCE}). best_local={best_local}")
                break

            poblacion = siguiente_generacion(poblacion, fits)

        # mejor individuo de esta seed
        fits = [fitness(ind) for ind in poblacion]
        idx_best = int(np.argmin(fits))
        ind_best = poblacion[idx_best]
        fit_best = fits[idx_best]

        if fit_best < mejor_fit:
            mejor_fit = fit_best
            mejor_seed = seed
            mejor_ind = ind_best

    print("\n==============================")
    print("  MEJOR SOLUCIÓN ENCONTRADA")
    print(f"  seed: {mejor_seed}")
    print(f"  fitness: {mejor_fit}")
    print("==============================\n")

    ok, cont = verificar_horas(mejor_ind)
    if ok:
        print("✅ Verificación horas: OK (todas las horas cuadran)")
    else:
        print("⚠️ Verificación horas: NO OK (revisa HORAS_CURSO)")
        # imprime solo las que fallan
        for c in cont:
            if cont[c] != HORAS_CURSO[c]:
                print(f"  {c}: asignadas={cont[c]} requeridas={HORAS_CURSO[c]}")

    solucion = individuo_a_horarios(mejor_ind)

    print("\n📌 MOSTRANDO TODOS LOS CICLOS (1..10) EN TABLAS (texto plano)\n")
    mostrar_todos(solucion)

    exportar_csv(solucion, OUTDIR)
    crear_zip(OUTDIR, ZIP_NAME)

    print(f"\n✅ CSV exportados en: {OUTDIR}")
    print(f"✅ ZIP creado: {ZIP_NAME}")
    return solucion

# Ejecutar
solucion_final = run()



[seed=277] Gen 1/500 best=680 global_best=680
[seed=277] Gen 50/500 best=480 global_best=480
[seed=277] Gen 100/500 best=300 global_best=300
[seed=277] Gen 150/500 best=220 global_best=220
[seed=277] Gen 200/500 best=200 global_best=200
[seed=277] Gen 250/500 best=160 global_best=160
[seed=277] Gen 300/500 best=160 global_best=160
[seed=277] Gen 350/500 best=140 global_best=140
[seed=277] Gen 400/500 best=100 global_best=100
[seed=277] Gen 450/500 best=100 global_best=100
[seed=277] Gen 500/500 best=80 global_best=80
[seed=354] Gen 1/500 best=55560 global_best=55560
[seed=354] Gen 50/500 best=32580 global_best=32580
[seed=354] Gen 100/500 best=25680 global_best=25680
[seed=354] Gen 150/500 best=25680 global_best=25680
[seed=354] Gen 200/500 best=25220 global_best=25220
[seed=354] Gen 250/500 best=25220 global_best=25220
[seed=354] Gen 300/500 best=25220 global_best=25220
[seed=354] Gen 350/500 best=25220 global_best=25220
[seed=354] Gen 400/500 best=25220 global_best=25220
[seed=354] G

,PERIODO,LUNES,MARTES,MIERCOLES,JUEVES,VIERNES
0,1,,ISI1102,,,ISI1106
1,2,ISI1101,ISI1102,ISI1105,,ISI1106
2,3,ISI1101,ISI1104,ISI1105,ISI1105,ISI1103
3,4,ISI1101,ISI1104,ISI1104,ISI1105,ISI1103
4,5,ISI1101,,ISI1104,,ISI1105
5,6,,,ISI1101,ISI1102,ISI1105
6,7,,,ISI1103,ISI1102,
7,8,,,ISI1103,ISI1104,
8,9,,,,,
9,10,,,,,


🟦 HORARIO CICLO 2 (solo cursos) 🟦


,PERIODO,LUNES,MARTES,MIERCOLES,JUEVES,VIERNES
0,1,ISI1203,,ISI1201,,ISI1205
1,2,ISI1203,ISI1201,ISI1201,ISI1205,ISI1205
2,3,ISI1203,ISI1201,ISI1204,ISI1205,ISI1204
3,4,,ISI1201,ISI1204,ISI1202,ISI1204
4,5,,ISI1203,,ISI1202,ISI1205
5,6,,ISI1203,ISI1204,,ISI1205
6,7,,ISI1206,ISI1202,,
7,8,,ISI1206,ISI1202,,
8,9,,,,,
9,10,,,,,


🟦 HORARIO CICLO 3 (solo cursos) 🟦


...

,PERIODO,LUNES,MARTES,MIERCOLES,JUEVES,VIERNES
0,1,,,,,
1,2,,,,,
2,3,,,,,
3,4,,,,,
4,5,,,,,
5,6,,,,,
6,7,,,,,
7,8,,,,,
8,9,,,,,
9,10,ISI2002,ISI2004,,,



✅ CSV exportados en: salida_horarios
✅ ZIP creado: horarios_finales.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[seed=277] Gen 1/500 best=640 global_best=640
[seed=277] Gen 50/500 best=560 global_best=560
[seed=277] Gen 100/500 best=460 global_best=460
[seed=277] Gen 150/500 best=460 global_best=460
⏹ Early stop seed=277 en Gen 176 (sin mejora 80). best_local=460
[seed=354] Gen 1/500 best=660 global_best=460
[seed=354] Gen 50/500 best=480 global_best=460
⏹ Early stop seed=354 en Gen 84 (sin mejora 80). best_local=480
[seed=431] Gen 1/500 best=620 global_best=460
[seed=431] Gen 50/500 best=400 global_best=400
[seed=431] Gen 100/500 best=400 global_best=400
⏹ Early stop seed=431 en Gen 122 (sin mejora 80). best_local=400

  MEJOR SOLUCIÓN ENCONTRADA
  seed: 431
  fitness: 400

✅ Verificación horas: OK (todas las horas cuadran)

📌 MOSTRANDO TODOS LOS CICLOS (1..10) EN TABLAS (texto plano)


🟦 HORARIO CICLO 1 (solo cursos) 🟦
PER LUNES           MARTES          MIERCOLES       JUEVES          VIERNES         
------------------------------------------------------------------------------------
1   ISI

In [2]:
%pwd

'/content'